In [19]:
! pip install h2o -q

In [20]:
import h2o 
from h2o.automl import H2OAutoML
import pandas as pd
import time
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score, precision_recall_fscore_support, \
    confusion_matrix

In [21]:
h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321 . connected.


H2O_cluster_uptime:,1 hour 7 mins
H2O_cluster_timezone:,Etc/UTC
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.34.0.3
H2O_cluster_version_age:,1 day
H2O_cluster_name:,H2O_from_python_unknownUser_7zcht2
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,2.866 Gb
H2O_cluster_total_cores:,2
H2O_cluster_allowed_cores:,2
H2O_cluster_status:,"locked, healthy"


In [22]:
dfLiarSementic = pd.read_csv("/content/drive/Shareddrives/[FYP] Fake News Detection/Results/LIAR/Liar_sementic.csv")
dfLiarSentiment = pd.read_csv("/content/drive/Shareddrives/[FYP] Fake News Detection/Results/LIAR/LIAR_predictions_emotions.csv")
dfLiarLexicon = pd.read_csv("/content/drive/Shareddrives/[FYP] Fake News Detection/Results/LIAR/Liar_lexicon.csv")

In [23]:
dff=dfLiarSentiment.merge(dfLiarSementic, how='inner', on='ID',suffixes=('_Sentiment', '_Sementic'))
df=dff.merge(dfLiarLexicon, how='inner', on='ID',suffixes=('', '_Lexicon'))
df=df.drop(df.columns[df.columns.str.contains('unnamed',case = False)],axis = 1)

In [24]:
df["qn_symbol_per_sentence"]=df["qn_symbol"]/df["num_sentences"]
df["num_exclamation_per_sentence"]=df["num_exclamation"]/df["num_sentences"]
df["url_count_per_sentence"]=df["url_count"]/df["num_sentences"]

In [25]:
sementic_features= [
                    'url_count',
                    'qn_symbol',
                    'num_chars',
                    'num_words',
                    'num_sentences',
                    'words_per_sentence',
                    'characters_per_word',
                    'punctuations_per_sentence',
                    'num_exclamation',
                    'get_sentiment_polarity',
                    'lexical_diversity',
                    'content_word_diversity',
                    'redundancy',
                    'noun',
                    'verb',
                    'adj',
                    'adv',
                    "qn_symbol_per_sentence",
                    "num_exclamation_per_sentence",
                    "url_count_per_sentence"
                    ]

LangMod_Features=   [
                     'fake_score', 
                    'true_score', 
                    'common_score'
                    ]

Sentiment_features= [
                     'highest_eight_label', 
                    'anger', 
                    'anticipation',
                    'disgust', 
                    'fear',
                    'joy', 
                    'sadness', 
                    'surprise', 
                    'trust'
                    ]
All_features=sementic_features+LangMod_Features+Sentiment_features

In [26]:
df=df.loc[df["lang"]=="en"]

In [27]:
df["label"]=df["label_Sentiment"]

In [28]:
df=df[All_features+["label","ID","split_Sentiment"]]

In [29]:
df["label"] = df["label"].replace(["true","mostly-true","half-true"],0)
df["label"] = df["label"].replace(["false","barely-true","pants-fire"],1)

In [30]:
df_train = df.loc[df["split_Sentiment"]=="train"][All_features+['label']]
df_test = df.loc[df["split_Sentiment"]=="test"][All_features+['label']]
df_valid = df.loc[df["split_Sentiment"]=="valid"][All_features+['label']]

In [31]:
hf_train = h2o.H2OFrame(df_train)
hf_valid = h2o.H2OFrame(df_valid)

cat_feat = ['label', 'highest_eight_label']
hf_train[cat_feat] = hf_train[cat_feat].asfactor()
hf_valid[cat_feat] = hf_valid[cat_feat].asfactor()

Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%


In [32]:
x = hf_train.columns
x.remove('label')

aml_h1n1 = H2OAutoML(
    max_models = 25, 
    seed = 10, 
    verbosity="info", 
    # exclude_algos = ["DeepLearning"],
    balance_classes = True,
    max_after_balance_size = 1.5,
    max_runtime_secs_per_model = 120,
    # nfolds=0
)

aml_h1n1.train(x = x, y = 'label', training_frame = hf_train, validation_frame=hf_valid)

AutoML progress: |
09:24:39.940: Project: AutoML_2_20211009_92439
09:24:39.940: User specified a validation frame with cross-validation still enabled. Please note that the models will still be validated using cross-validation only, the validation frame will be used to provide purely informative validation metrics on the trained models.
09:24:39.943: Setting stopping tolerance adaptively based on the training frame: 0.009925833339709303
09:24:39.943: Build control seed: 10
09:24:39.943: training frame: Frame key: AutoML_2_20211009_92439_training_py_339_sid_93f2    cols: 33    rows: 10150  chunks: 1    size: 1437309  checksum: 3684620907594347943
09:24:39.944: validation frame: Frame key: py_340_sid_93f2    cols: 33    rows: 1271  chunks: 1    size: 162180  checksum: -3033394481074263593
09:24:39.944: leaderboard frame: NULL
09:24:39.944: blending frame: NULL
09:24:39.944: response column: label
09:24:39.944: fold column: null
09:24:39.944: weights column: null
09:24:39.944: Loading exec

,,0,1,Error,Rate
0,0,3994.0,1643.0,0.2915,(1643.0/5637.0)
1,1,890.0,3479.0,0.2037,(890.0/4369.0)
2,Total,4884.0,5122.0,0.2531,(2533.0/10006.0)



Maximum Metrics: Maximum metrics at their respective thresholds


,metric,threshold,value,idx
0,max f1,0.432692,0.733116,215.0
1,max f2,0.368893,0.829983,287.0
2,max f0point5,0.471899,0.743614,169.0
3,max accuracy,0.461168,0.761143,181.0
4,max precision,0.930802,1.000000,0.0
5,max recall,0.225016,1.000000,383.0
6,max specificity,0.930802,1.000000,0.0
7,max absolute_mcc,0.446434,0.510804,198.0
8,max min_per_class_accuracy,0.441276,0.753947,204.0
9,max mean_per_class_accuracy,0.446434,0.755940,198.0



Gains/Lift Table: Avg response rate: 43.66 %, avg score: 43.70 %


...

,,0,1,Error,Rate
0,0,573.0,5139.0,0.8997,(5139.0/5712.0)
1,1,213.0,4225.0,0.048,(213.0/4438.0)
2,Total,786.0,9364.0,0.5273,(5352.0/10150.0)



Maximum Metrics: Maximum metrics at their respective thresholds


,metric,threshold,value,idx
0,max f1,0.327027,0.612230,330.0
1,max f2,0.111925,0.795312,398.0
2,max f0point5,0.424952,0.517254,223.0
3,max accuracy,0.495834,0.581773,136.0
4,max precision,0.869973,1.000000,0.0
5,max recall,0.111925,1.000000,398.0
6,max specificity,0.869973,1.000000,0.0
7,max absolute_mcc,0.434716,0.131284,211.0
8,max min_per_class_accuracy,0.437684,0.563375,207.0
9,max mean_per_class_accuracy,0.434716,0.566157,211.0



Gains/Lift Table: Avg response rate: 43.72 %, avg score: 43.73 %


,group,cumulative_data_fraction,lower_threshold,lift,cumulative_lift,response_rate,score,cumulative_response_rate,cumulative_score,capture_rate,cumulative_capture_rate,gain,cumulative_gain,kolmogorov_smirnov
0,1,0.010049,0.638348,1.479866,1.479866,0.647059,0.700296,0.647059,0.700296,0.014872,0.014872,47.986639,47.986639,0.008569
1,2,0.020000,0.603646,1.381297,1.430825,0.603960,0.618842,0.625616,0.659769,0.013745,0.028616,38.129744,43.082470,0.015311
2,3,0.030049,0.587569,1.435022,1.432228,0.627451,0.595446,0.626230,0.638258,0.014421,0.043037,43.502196,43.222837,0.023079
3,4,0.040000,0.577385,1.200144,1.374493,0.524752,0.582514,0.600985,0.624391,0.011942,0.054980,20.014367,37.449301,0.026618
4,5,0.050049,0.568319,1.345333,1.368638,0.588235,0.572406,0.598425,0.613953,0.013520,0.068499,34.533309,36.863807,0.032785
5,6,0.100000,0.535002,1.245030,1.306895,0.544379,0.549525,0.571429,0.581770,0.062190,0.130689,24.503015,30.689500,0.054534
6,7,0.150049,0.516193,1.134529,1.249402,0.496063,0.525128,0.546290,0.562877,0.056782,0.187472,13.452893,24.940192,0.066498
7,8,0.200000,0.500163,1.199920,1.237044,0.524655,0.508178,0.540887,0.549216,0.059937,0.247409,19.992036,23.704371,0.084243
8,9,0.300000,0.475932,1.135647,1.203245,0.496552,0.487727,0.526108,0.528719,0.113565,0.360973,13.564669,20.324470,0.108347
9,10,0.400000,0.455430,1.104101,1.178459,0.482759,0.465545,0.515271,0.512926,0.110410,0.471384,10.410095,17.845877,0.126846


In [33]:
hf_test = h2o.H2OFrame(df_test)

cat_feat = ['label', 'highest_eight_label']
hf_test[cat_feat] = hf_test[cat_feat].asfactor()

Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%


In [34]:
pred=aml_h1n1.leader.predict(hf_test)

predictions = pred.as_data_frame()

predictions.head()

stackedensemble prediction progress: |███████████████████████████████████████████| (done) 100%


,predict,p0,p1
0,1,0.493928,0.506072
1,1,0.553327,0.446673
2,1,0.452590,0.547410
3,1,0.459324,0.540676
4,1,0.603431,0.396569


In [35]:
y_pred = predictions['predict'].tolist()
y_true = df_test['label'].tolist()

accuracy_score(y_true, y_pred)

0.43626286619160726